### モジュール読み込み

In [1]:
import os

import numpy as np
import pandas as pd

from trackml.dataset import load_event,  load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event

%matplotlib inline

import sys
sys.path.append("../lib/")

import hough_clusterer as hc
from hyperopt import fmin, tpe, hp, rand

### データ読み込み

In [2]:
path_to_train = '../path_to_train'
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join(path_to_train, event_prefix))

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event_prefix, mem_bytes / 2**20))

event000001000 memory usage 18.46 MB


### モデル

In [12]:
parameter_space = {
    'N_bins_r0inv': hp.choice('N_bins_r0inv', [100, 200, 300, 400]),
    'N_bins_gamma': hp.choice('N_bins_gamma', [100, 300, 500, 700]),
    'N_theta': hp.choice('N_theta', [100, 300, 500, 700]),
    'min_hits': hp.quniform('min_hits', 5, 15, 1)
}

def function(args):
    print(args)
    model = hc.Clusterer(**args)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)
    return -score

### スコア関数

In [13]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

### ハイパーパラメータ最適化

In [14]:
best = fmin(function, parameter_space, algo=tpe.suggest, max_evals=10)
print("best parameters")
print(best)
model = hc.Clusterer(**best)
print(model)

{'N_bins_gamma': 300, 'N_bins_r0inv': 100, 'N_theta': 500, 'min_hits': 11.0}
Your score:  0.10885895734525605
{'N_bins_gamma': 100, 'N_bins_r0inv': 200, 'N_theta': 300, 'min_hits': 14.0}
Your score:  0.03868395594026128
{'N_bins_gamma': 300, 'N_bins_r0inv': 100, 'N_theta': 500, 'min_hits': 8.0}
Your score:  0.09081057059767003
{'N_bins_gamma': 100, 'N_bins_r0inv': 400, 'N_theta': 300, 'min_hits': 8.0}
Your score:  0.08265108558464049
{'N_bins_gamma': 500, 'N_bins_r0inv': 400, 'N_theta': 100, 'min_hits': 9.0}
Your score:  0.0386462749347377
{'N_bins_gamma': 500, 'N_bins_r0inv': 200, 'N_theta': 500, 'min_hits': 9.0}
Your score:  0.1403401478160794
{'N_bins_gamma': 500, 'N_bins_r0inv': 400, 'N_theta': 500, 'min_hits': 15.0}
Your score:  0.01219600740457967
{'N_bins_gamma': 100, 'N_bins_r0inv': 200, 'N_theta': 300, 'min_hits': 5.0}
Your score:  0.016810617120154896
{'N_bins_gamma': 500, 'N_bins_r0inv': 200, 'N_theta': 700, 'min_hits': 7.0}
Your score:  0.10893187665666071
{'N_bins_gamma': 